In [9]:
# This code takes in the .out files generated from rmsd.tcl and computes the BEST RMSD fits 
# between WNT1/3a/5a and WNT8a. Output is a csv file containing a three column table: 1st column
# contains each WNT1/3a/5a index and the 2nd column contains the WNT8A residue index that best maps 
# to the WNT1/3a/5a residue. The third column is the RMSD value of that fit. Example below:
#
# WNT1/3a/5a INDEX  | WNT8A INDEX | RMSD FIT
#       22          |     55      |    2.9
# .............................................
#
##############
# PURPOSE
##############
# .out files generated contain all possible RMSD fits between WNT1/3a/5a and WNT8a.
# The first column of each row is the index of the residue being fit to on WNT1/3a/5a
# Then, the RMSD fit to each possible residue on WNT8a is output in subsequent columns
#
##############
# HOW RMSD FITTING WORKS
##############
#
# We consider RMSD fits between N residue sequences on WNT1/3a/5a and WNT8.
#
# For Example: We fit residue 60 on WNT1 (residues 40-80) to all possible 20 residue sequences on WNT8a
#
# We take the residue range as follows:
#
# X = Current WNT residue
# N = Sequence Length cutoff.
# A, B = Start/END idx of WNT
# 
#
# IF X - N < A:
#      RANGE = [X, X+2N]
# IF X + N > B:
#      RANGE = [X-2N, X]
# ELSE:
#      RANGE = [X-N, X+N]


import numpy as np
import pandas as pd

# Selector - change SYSTEM variable to generate mappings for each system (1/3a/5a/8a)
SYSTEM = "8a"
if SYSTEM == "1":
    fitwnt = [32,369]
if SYSTEM == "3a":
    fitwnt = [19,352]
if SYSTEM == "5a":
    fitwnt = [44,380]
if SYSTEM == "8a":
    fitwnt = [22,337]


In [10]:
refwnt = [22,337] # Indeces of wnt8a (protein we are mapping to)

offset = 20 # Preselected offset for rmsd fitting (20 = rmsd fitting done over ranges of 20 residues)

align_res100 = np.ones((fitwnt[1] - fitwnt[0] + 1, refwnt[1] - refwnt[0] + offset + 1)) * 500
print(align_res100.shape)

# Read in the mappings generated by tcl script
with open(f"output/{SYSTEM}_to_8a.out", "r") as file:
    curr_line = 0
    for line in file:
        
        sp = line.split()
        #print(c, len(sp))
        align_res100[curr_line,0:len(sp)] = sp
        curr_line = curr_line + 1
        #print(curr_line, sp[0])

hashmap= []
rmsd_cutoff = 4.5 # RMSD Cutoff, if best rmsd fit is greater than rmsd_cutoff -> mapping idx is set to -1 (no fit)

# For each pairing, check to see what the RMSD of the associated fit was
# Find the minimum RMSD fitting value -> assign idx as best fit/mapping
for i in range(len(align_res100)):
        fit_index = fitwnt[0]+i
        ref_index_at_min_rmsd = np.argmin(align_res100[i,1:])+align_res100[i,0]
        min_rmsd = align_res100[i,1+np.argmin(align_res100[i,1:])]
        if min_rmsd > rmsd_cutoff:
            hashmap.append([fit_index, -1, min_rmsd])
        else:
            hashmap.append([fit_index, ref_index_at_min_rmsd, min_rmsd])
        
# Output the mapping as a CSV file
hashmap = np.array(hashmap)
hashmap_df = pd.DataFrame(hashmap, columns=[SYSTEM,'8a','rmsd'])
hashmap_df[SYSTEM] = hashmap_df[SYSTEM].astype(int)
hashmap_df["8a"] = hashmap_df["8a"].astype(int)
hashmap_df.to_csv(f"output/{SYSTEM}_to_8a_map.csv", index=False)
hashmap_df

(316, 336)


,8a,8a,rmsd
0,22,22,0.000000
1,23,23,0.000002
2,24,24,0.000002
3,25,25,0.000003
4,26,26,0.000003
...,...,...,...
311,333,333,0.000018
312,334,334,0.000019
313,335,335,0.000021
314,336,336,0.000022


In [11]:
import matplotlib.pyplot as plt
plt.plot(hashmap_df[SYSTEM], hashmap_df["rmsd"])
plt.xlabel("Wnt3A Residue ID", fontsize=20)
plt.ylabel(f"Minimum RMSD ($\AA$)", fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.savefig(f"{SYSTEM}_to_8a_map.png", dpi=400)

<>:4: SyntaxWarning: invalid escape sequence '\A'
<>:4: SyntaxWarning: invalid escape sequence '\A'
/var/folders/p_/mmkyzkc1655_rf8nypyzqx7r0000gn/T/ipykernel_12480/3819731559.py:4: SyntaxWarning: invalid escape sequence '\A'
  plt.ylabel(f"Minimum RMSD ($\AA$)", fontsize=20)


ModuleNotFoundError: No module named 'matplotlib'